In [1]:
# 2025 08 28

# 아이템 기반 협업필터링 실습
# ml-latest-small movies 사용

In [2]:
import pandas as pd
import numpy as np

movies = pd.read_csv('./data/ml-latest-small/movies.csv')
ratings = pd.read_csv('./data/ml-latest-small/ratings.csv')

print(movies.shape)
print(ratings.shape)

# (9742, 3)       --> (행, 열)
# (100836, 4)     --> (행, 열)

(9742, 3)
(100836, 4)


In [3]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
# 1. 사용자-아이템 정렬 데이터를 아이템-사용자 정렬 데이터로 변환
# 2. 아이템간의 코사인유사도로 아이템 유사도 산출
# 3. 사용자가 관람(구매)하지 않은 아이템 중에서 아이템간 유사도를 반영한 예측점수 계산
# 4. 예측 점수가 높은 순으로 아이템 추천

In [6]:
### 1. 사용자가-아이템 정렬 데이터를 아이템-사용자 정렬 데이터로 변환

In [7]:
ratings[['userId', 'movieId', 'rating']]
ratings_matrix = ratings.pivot_table('rating', index='userId', columns='movieId')
ratings_matrix.head()

# 행 : user_id   컬럼 : movieId

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# title 컬럼을 얻기 위해 movies와 ratings를 movieId로 조인 수행

# movies => movieId 별로 영화title과 genres르 가지고 있는 데이터프레임
# ratings => userId별로 movieId, rating, timestamp를 가지고 있는 데이터프레임

rating_movies = pd.merge(ratings, movies, on='movieId')  
rating_movies.head()

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [9]:
# 조인한 결과로 pivot 테이블 만들기

# index(행번호) = 'userId' , columns(열 이름)='title)   -->  해당 결과로 pivot 테이블 생성
ratings_matrix = rating_movies.pivot_table('rating', index='userId', columns='title')

# NaN 값을 모두 0으로 반환! (oracle로 따지면 nvl)
ratings_matrix = ratings_matrix.fillna(0)

ratings_matrix.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# transpose()   :  행과 열을 서로 바꿈
ratings_matrixT = ratings_matrix.transpose()

ratings_matrixT.head(3)

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
'Hellboy': The Seeds of Creation (2004),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
'Round Midnight (1986),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
ratings_matrixT.shape      # (9719, 610) : title(행) userId(열)

(9719, 610)

In [12]:
# 2. 아이템간의 코사인 유사도로 아이템 유사도 산출
# 영화와 영화들간의 유사도 산출

In [13]:
from sklearn.metrics.pairwise import cosine_similarity   # 코사인유사도


# cosine_similarity() 함수를 통해 ratings_matrixT 즉, 아이템 사용자 평점행렬의 코사인 유사도 값을 구한다
item_sim = cosine_similarity(ratings_matrixT, ratings_matrixT)

# cosine_similarity()로 반환된 넘파이 행렬을 영화명을 매핑하여 DataFrame으로 변환
item_sim_df = pd.DataFrame(data=item_sim, index= ratings_matrix.columns , columns= ratings_matrix.columns)

print(item_sim_df.shape)    # (9719, 9719)

(9719, 9719)


In [14]:
item_sim_df.head(3)

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.141653,0.0,...,0.0,0.342055,0.543305,0.707107,0.0,0.0,0.139431,0.327327,0.0,0.0
'Hellboy': The Seeds of Creation (2004),0.0,1.000000,0.707107,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0
'Round Midnight (1986),0.0,0.707107,1.000000,0.0,0.0,0.0,0.176777,0.0,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0


In [15]:
# 유사도 높은 순으로 6개만 추출

item_sim_df["Godfather, The (1972)"].sort_values(ascending=False) [  : 6]

# title
# Godfather, The (1972)                        1.000000
# Godfather: Part II, The (1974)               0.821773
# Goodfellas (1990)                            0.664841
# One Flew Over the Cuckoo's Nest (1975)       0.620536
# Star Wars: Episode IV - A New Hope (1977)    0.595317
# Fargo (1996)                                 0.588614
# Name: Godfather, The (1972), dtype: float64

title
Godfather, The (1972)                        1.000000
Godfather: Part II, The (1974)               0.821773
Goodfellas (1990)                            0.664841
One Flew Over the Cuckoo's Nest (1975)       0.620536
Star Wars: Episode IV - A New Hope (1977)    0.595317
Fargo (1996)                                 0.588614
Name: Godfather, The (1972), dtype: float64

In [16]:
# 본인거 빼고 6개 추출
item_sim_df["Inception (2010)"].sort_values(ascending=False) [  : 6]


# title
# Inception (2010)                 1.000000
# Dark Knight, The (2008)          0.727263
# Inglourious Basterds (2009)      0.646103
# Shutter Island (2010)            0.617736
# Dark Knight Rises, The (2012)    0.617504
# Fight Club (1999)                0.615417
# Name: Inception (2010), dtype: float64

title
Inception (2010)                 1.000000
Dark Knight, The (2008)          0.727263
Inglourious Basterds (2009)      0.646103
Shutter Island (2010)            0.617736
Dark Knight Rises, The (2012)    0.617504
Fight Club (1999)                0.615417
Name: Inception (2010), dtype: float64

In [17]:
# 3. 사용자가 관람(구매)하지 않은 아이템 중에서 아이템간 유사도를 반영한 예측 점수 계산

In [18]:
# 예측 평점
def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr) / np.array([np.abs(item_sim_arr).sum(axis=1)])   # 카페 슬라이드 내 고익 참고
    return ratings_pred

In [19]:
ratings_matrix.head(3)    # 사용자별로 각각의 아이템(타이틀)들에 대해 평점 구함

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
# 예측 평점 함수 호출
ratings_pred = predict_rating(ratings_matrix.values, item_sim_df.values)     # (사용자에 대한 평점점수, 유사도 산출값)

# index = userId정보,   columns = title정보
ratings_pred_matrix = pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)

print(ratings_pred_matrix.shape)       # (610, 9719)
ratings_pred_matrix.head(3)

(610, 9719)


title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.070345,0.577855,0.321696,0.227055,0.206958,0.194615,0.249883,0.102542,0.157084,0.178197,...,0.113608,0.181738,0.133962,0.128574,0.006179,0.212070,0.192921,0.136024,0.292955,0.720347
2,0.018260,0.042744,0.018861,0.000000,0.000000,0.035995,0.013413,0.002314,0.032213,0.014863,...,0.015640,0.020855,0.020119,0.015745,0.049983,0.014876,0.021616,0.024528,0.017563,0.000000
3,0.011884,0.030279,0.064437,0.003762,0.003749,0.002722,0.014625,0.002085,0.005666,0.006272,...,0.006923,0.011665,0.011800,0.012225,0.000000,0.008194,0.007017,0.009229,0.010420,0.084501


In [21]:
## 가중치 평점 부여 뒤에 예측 성능 평가 MSE를 구함

In [22]:
# 예측 평점을 구했으므로, 원래 원본 데이터의 평점과 비교
from sklearn.metrics import mean_squared_error

# 사용자가 평점을 부여한 영화에 대해서만 예측 성능 평가 MSE를 구함
def get_mse(pred, actual):
    # 0은 원래 NaN이었는데, 위에서 모든 NaN을 0으로 바꾸는 작업을 했음.
    # Ignore nonzero terms : 0을 제외하고 예측성능평가를 구하자
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()
    return mean_squared_error(pred, actual)         # 구한 값을 갖고 다시 return 


print('아이템 기반 모든 인접 이웃 MSE : ', get_mse(ratings_pred, ratings_matrix.values))
# 아이템 기반 모든 인접 이웃 MSE :  9.895354759094706

아이템 기반 모든 인접 이웃 MSE :  9.895354759094706


In [23]:
# top-n 유사도 기반의 예측 평점 및 MSE 계산

In [24]:
def predict_rating_topsim(ratings_arr, item_sim_arr, n=20): # 매개변수(평점, 타이틀) top-20개
    # 사용자-아이템 평점 행렬 크기만큼 0으로 채운 예측 행렬 초기화
    pred = np.zeros(ratings_arr.shape)

    # 사용자-아이템 평점 행렬의 열 크기만큼 loop 수행
    for col in range(ratings_arr.shape[1]):
        # 유사도 행렬에서 유사도가 큰순(-1 역순)으로 n개 데이터 행렬의 index 반환
        top_n_items = [np.argsort(item_sim_arr[ :, col])[ :-n-1:-1]]

        # 개인화된 예측 평점을 계산
        for row in range(ratings_arr.shape[0]):
            pred[row, col] = item_sim_arr[col, : ][top_n_items].dot(ratings_arr[row, :][top_n_items].T)
            pred[row, col] /= np.sum(np.abs(item_sim_arr[col, : ][top_n_items]))
    return pred

In [ ]:
# 함수 호출
ratings_pred = predict_rating_topsim(ratings_matrix.values, item_sim_df.values, n=20)
print('아이템 기반 top-20 인접 이웃 MSE : ', get_mse(ratings_pred, ratings_matrix.values))

C:\Users\ict1\AppData\Local\Temp\ipykernel_18948\1666190108.py:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  pred[row, col] = item_sim_arr[col, : ][top_n_items].dot(ratings_arr[row, :][top_n_items].T)


In [ ]:
ratings_pred = matrix= pd.DataFrame(data=ratings_pred, index=ratings_matrix.index, columns=ratings_matrix.columns)

In [ ]:
user_rating_id = ratings_matrix.loc[9,  : ]      # 특정사용자 9번이 좋아하는 영화를 평점순으로 10건 추출
user_rating_id[user_rating_id > 0].sort_values(ascending=False)[:10]

In [ ]:
# 9번 사용자가 관람하지 않은 영화 중에서 아이템 기반의 인접 이웃 협업 필링으로 영화 추천
def get_unseen_movies(ratings_matrix, userId):
    # userId로 입력받은 사용자의 모든 영화정보를 추출하여 Series로 반환함
    # 반환된 user_rating은 영화명(title) 을 index로 가지는 Series 객체다.
    user_rating = ratings_matrix.loc[userId, :]

    # user_rating > 0 ==> 기존에 관람한 영화
    # 대상 index를 추출해서 list 객체로 만듦
    already_seen = user_rating[user_rating > 0].index.tolist()

    # 모든 영화명을 list 객체로 만듦
    movies_list = ratings_matrix.columns.tolist()

    # movie가 already_seen에 없다면, 반복문을 통해 본 적 없는 영화-> movie를 추출
    unseen_list = [movie for movie in movies_list if movie not in already_seen]

    return unseen_list

In [ ]:
# 4. 예측 점수가 높은 순으로 아이템 추천

In [ ]:
# 아이템 기반 유사도로 평점이 부여된 데이터셋에서 해당 사용자가 관람하지 않은 영화들의 예측 평점이 가장 높은 영화를 추천

In [ ]:
def recomm_movie_by_userid(pred_df, userId, unseen_list, top_n=10):
    # 예측평점 CataFrame에서 userId, unseen_list로 들어온 영화명 컬럼을 추출하여
    # 가장 예측 평점이 높은 순으로 정렬
    recomm_movies = pred_df.loc[userId, unseen_list].sort_values(ascending=False)[ : top_n]
    return recomm_movies

# 함수호출 : 사용자가 관람하지 않은 영화명 추출
unseen_list= get_unseen_movies(ratings_matrix, 9)

# 함수호출 : 아이템 기반의 인접 이웃 협업 필터링으로 영화 추천
recomm_movies = recomm_movie_by_userid(ratings_pred_matrix, 9, unseen_list, top_n=10)

# 평점 데이터를 DataFrame으로 생성
recomm_movies = pd.DataFrame(data=recomm_movies.values, index=recomm_movies.index, columns=['pred_score'])
recomm_movies